In [76]:
import json
import string
import random
import re
import urllib.request
from nltk.stem import WordNetLemmatizer
import numpy as np
# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs
import nltk
# Descargar el diccionario
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import sys
#import gradio as gr
import pandas as pd

In [77]:
!{sys.executable} -m pip install gradio --quiet

##OWN SIMILARITY

In [78]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

In [79]:
def one_hot_encoding (corpus):
    
    words_set = {}
    words_set = set(words_set)
    
    for k in corpus:
        
        words = set(k.split(" "))
        
        for i in words :
            
            words_set.add(i)
        
    
    one_h_encod = np.zeros((len(corpus),len(words_set)))
    
    tf_ = np.zeros((len(corpus),len(words_set)))
    
    words_list = list(words_set)
    
    
    for q in range(0,len(corpus)):
        
        
        words_  = list(corpus[q].split(" "))
        
        for h in words_:
                    
            one_h_encod [q][words_list.index(h)] = 1
            tf_[q][words_list.index(h)] = tf_[q][words_list.index(h)] + 1
    
    return pd.DataFrame(one_h_encod,columns = words_list) , pd.DataFrame(tf_ , columns = words_list)

##TEACHER'S SIMILARITY

In [80]:
def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [81]:
def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [82]:
def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias

    corpus.append(user_input)
    df_ohe, df_tf = one_hot_encoding(corpus)

    ####### IDF #######

    idf = np.log10(len(corpus) / df_ohe.sum(axis= 0 )) 

    df_idf = pd.DataFrame(idf).T

    ####### IDF #######

    ####### TF_IDF #######

    tf_idf = df_tf.copy()

    cont = 0

    for j in df_tf.columns:
    
      tf_idf[j] = df_tf[j]*idf[cont]
      cont = cont + 1 

    ####### TF_IDF #######


    ####### COSINE SIMILARITY #######

    cosine_m = np.zeros((len(corpus),len(corpus)))

    similitud = 0
    mayor = 0
    response = " "
    for t in range(len(corpus)):
                
        a = np.array(tf_idf.iloc[t])
        b = np.array(tf_idf.iloc[-1])

        similitud =  (cosine_similarity(a,b))
        if similitud >= mayor:
          if user_input != corpus[t]:
            mayor = similitud
            response = corpus[t]
          

    ####### COSINE SIMILARITY #######
    
  
    return response

In [83]:
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


##TOPICS:
* HOT DOG
* FAST FOOD
* COCA COLA
* FOOD
* OBESITY
* OVERWEIGHT
* SEDENTARY LIFESTYLE

##WE ONLY SELECT 3 TOPICS RANDOMLY, IT CHANGES IN EACH EXECUTION

In [84]:
topics = ["https://en.wikipedia.org/wiki/Hot_dog","https://en.wikipedia.org/wiki/Fast_food","https://en.wikipedia.org/wiki/Coca-Cola","https://en.wikipedia.org/wiki/Food","https://en.wikipedia.org/wiki/Obesity","https://en.wikipedia.org/wiki/Overweight","https://en.wikipedia.org/wiki/Sedentary_lifestyle"]

topics = random.sample(topics, 3)


full_article = ""

for i in topics:
  raw_html = urllib.request.urlopen(i)
  raw_html = raw_html.read()

  article_html = bs.BeautifulSoup(raw_html, 'lxml')

  article_paragraphs = article_html.find_all('p')

  article_text = ''

  for para in article_paragraphs:
      article_text += para.text

  article_text = article_text.lower()
  full_article = full_article + article_text

In [85]:
print(topics)

['https://en.wikipedia.org/wiki/Hot_dog', 'https://en.wikipedia.org/wiki/Coca-Cola', 'https://en.wikipedia.org/wiki/Obesity']
['https://en.wikipedia.org/wiki/Food', 'https://en.wikipedia.org/wiki/Sedentary_lifestyle', 'https://en.wikipedia.org/wiki/Fast_food']


In [86]:
text = re.sub(r'\[[0-9]*\]', ' ', full_article)
text = re.sub(r'\s+', ' ', text)

In [87]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [ ]:
import sys
import gradio as gr

!{sys.executable} -m pip install gradio --quiet


def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://55672.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Q: overweight
A: a study done in the city of jeddah has shown that current fast-food habits are related to the increase of overweight and obesity among adolescents in saudi arabia.


In [ ]:
topics